In [26]:
import numpy as np
from scipy.special import ellipk, ellipe

def compute_greens_function(r, r_target, z, z_target):
    """
    Computes the Green's function G.
    """
    z_d = z - z_target
    a = r**2 + r_target**2 + z_d**2
    b = 2 * r * r_target
    m = (4 * r * r_target) / ((r + r_target)**2 + z_d**2)
    print(f"[m]: {m}")
    
    # Ensure m is in the valid range
    if m < 0 or m >= 1:
        raise ValueError("The modulus m is out of the valid range [0, 1).")
    
    # Compute elliptic integral K(m)
    K_m = ellipk(m)
    print(f"[K_m]: {K_m}")
    # Compute Green's function G
    G = r * (4 / np.sqrt(a + b)) * K_m
    return G

def compute_greens_function_derivative_normal(r, r_target, z, z_target, nr, nz):
    """
    Computes the directional derivative of the Green's function (∂G/∂n).
    """
    z_d = z - z_target
    a = r**2 + r_target**2 + z_d**2
    b = 2 * r * r_target
    m = (4 * r * r_target) / ((r + r_target)**2 + z_d**2)
    # print(f"[m]: {m}")
    
    # Ensure m is in the valid range
    if m < 0 or m >= 1:
        raise ValueError("The modulus m is out of the valid range [0, 1).")
    
    # Compute elliptic integrals K(m) and E(m)
    K_m = ellipk(m)
    E_m = ellipe(m)
    print(f"[E_m]: {E_m}")

    # Compute the derivative of Green's function ∂G/∂n
    term1 = 2 * nr / np.sqrt(a + b)
    term2 = r * (nr * (r - r_target) + nz * z_d) / ((a - b) * np.sqrt(a + b))
    print(f"[term1 = ]", term1)
    print(f"[term2 = ]", term2)
    dG_dn = term1 * (E_m - K_m) + term2 * E_m
    
    return dG_dn

def integrate_k1_k2(r1, z1, r2, z2, r_target, z_target, nr, nz, num_points=7):
    """
    Computes [k1] and [k2] using Gaussian quadrature.
    """
    # Define Gaussian quadrature points and weights for 7-point rule
    # Define Gaussian quadrature points and weights for 8-point rule
    gauss_points = [
        -0.96028985649753623168,
        -0.79666647741362673959,
        -0.52553240991632898582,
        -0.18343464249564980494,
         0.18343464249564980494,
         0.52553240991632898582,
         0.79666647741362673959,
         0.96028985649753623168
    ]
    gauss_weights = [
        0.10122853629037625915,
        0.22238103445337447054,
        0.31370664587788728734,
        0.36268378337836209114,
        0.36268378337836209114,
        0.31370664587788728734,
        0.22238103445337447054,
        0.10122853629037625915
    ]

    
    # Element length and scale factor
    element_length = np.sqrt((z2 - z1)**2 + (r2 - r1)**2)
    scale = element_length / 2.0
    
    k11 = 0.0
    k12 = 0.0
    k21 = 0.0
    k22 = 0.0
    
    for xi, weight in zip(gauss_points, gauss_weights):
        # Map xi to physical coordinates
        z = z1 * (1 - xi) / 2.0 + z2 * (1 + xi) / 2.0
        r = r1 * (1 - xi) / 2.0 + r2 * (1 + xi) / 2.0
        print(f"(z, r): {z, r}")
        
        # Compute Green's function and its derivative
        G = compute_greens_function(r, r_target, z, z_target)
        dG_dn = compute_greens_function_derivative_normal(r, r_target, z, z_target, nr, nz)
        print(f"[G * r]: {G}")
        print(f"[dG_dn * r]: {dG_dn}")
        
        # Compute shape functions
        N1 = (1 - xi) / 2.0
        N2 = (1 + xi) / 2.0
        
        # Integrate [k1] and [k2]
        k11 += weight * (N1 * dG_dn)
        k12 += weight * (N2 * dG_dn)
        k21 += weight * (N1 * G)
        k22 += weight * (N2 * G)
    
    # Scale by element length
    k11 *= scale
    k12 *= scale
    k21 *= scale
    k22 *= scale
    
    return k11, k12, k21, k22

# Example usage
r1, z1 = 2.0, 5.0  # Node 1 coordinates (xi)
r2, z2 = 3.0, 6.0  # Node 2 coordinates (xi+1)
r_target, z_target = 2.0, 5.0  # Target point coordinates (ri, zi)
nr, nz = 1.0, 0.0  # Outward normal vector components

k11, k12, k21, k22 = integrate_k1_k2(r1, z1, r2, z2, r_target, z_target, nr, nz)

print(f"[k11]: {k11}")
print(f"[k12]: {k12}")
print(f"[k21]: {k21}")
print(f"[k22]: {k22}")

(z, r): (5.019855071751231, 2.019855071751232)
[m]: 0.9999512087964747
[K_m]: 6.350339882930864
[E_m]: 1.000142723320839
[term1 = ] 0.4975243058646313
[term2 = ] 12.653278279595684
[G * r]: 12.763255920069415
[dG_dn * r]: 9.993231069417348
(z, r): (5.1016667612931865, 2.1016667612931865)
[m]: 0.9987719940059069
[K_m]: 4.738624313808472
[E_m]: 1.0026028570293986
[term1 = ] 0.4874569337499555
[term2 = ] 2.5191911844959765
[G * r]: 9.709176189860266
[dG_dn * r]: 0.7045987152333713
(z, r): (5.237233795041835, 2.2372337950418357)
[m]: 0.9937503021111119
[K_m]: 3.928489028565047
[E_m]: 1.0107200685250333
[term1 = ] 0.4712680023463277
[term2 = ] 1.1110734678264862
[G * r]: 8.283900327499339
[dG_dn * r]: -0.25206689756824696
(z, r): (5.408282678752175, 2.408282678752175)
[m]: 0.9829900291146794
[K_m]: 3.4336699841011726
[E_m]: 1.0249906293548519
[term1 = ] 0.45175797440347154
[term2 = ] 0.6661811542367906
[G * r]: 7.471397404990393
[dG_dn * r]: -0.4053106657421489
(z, r): (5.591717321247826, 2

In [38]:
import numpy as np

def gauss_quadrature_8_point(f):
    # Define 8-point Gaussian quadrature points and weights
    gauss_points = [
        -0.96028985649753623168,
        -0.79666647741362673959,
        -0.52553240991632898582,
        -0.18343464249564980494,
         0.18343464249564980494,
         0.52553240991632898582,
         0.79666647741362673959,
         0.96028985649753623168
    ]
    gauss_weights = [
        0.10122853629037625915,
        0.22238103445337447054,
        0.31370664587788728734,
        0.36268378337836209114,
        0.36268378337836209114,
        0.31370664587788728734,
        0.22238103445337447054,
        0.10122853629037625915
    ]
    
    # Transform Gaussian points from [-1, 1] to [0, 1]
    transformed_points = [(1 + xi) / 2 for xi in gauss_points]
    transformed_weights = [w / 2 for w in gauss_weights]

    # Define Logarithmic Gaussian quadrature points and weights for 8-point rule
    log_gauss_points = [
        0.0133202441608924650122526725243,
        0.0797504290138949384098277291424,
        0.197871029326188053794476159516,
        0.354151399451909419671463603538,
        0.529458575234917277706149699996,
        0.701814529939099963837152670310,
        0.849379320441106676048309202301,
        0.953326450056359788767379678514
    ]
    
    log_gauss_weights = [
        0.16441660472800286831472568326,
        0.23752561002330602050134856196,
        0.22684198443191912636887804029,
        0.17575407900607024498880562120,
        0.11292403024675905185500044208,
        0.057872210717782072399527967294,
        0.020979073742132978043461524115,
        0.0036864071040276190133523212764
    ]

    
    # Compute the integral
    integral = 0.0
    for xi, w in zip(log_gauss_points, log_gauss_weights):
        integral += w * f(xi)
        # * np.log(1 / xi)
    
    return integral

# Define your function f(x) here
def f(x):
    return x**2 + 3 * x**3  # Example: Replace this with your desired function

# Compute the integral
result = gauss_quadrature_8_point(f)
print(f"Result of the integral: {result}")

Result of the integral: 0.29861027326515804


In [1]:
import numpy as np
from scipy.special import ellipk, ellipe

# Constants for logarithmic splitting
b0 = 1.38629436112  # Coefficient for logarithmic part

# Define logarithmic Gaussian quadrature points and weights for 8-point rule
log_gauss_points = [
    0.0133202441608924650122526725243,
    0.0797504290138949384098277291424,
    0.197871029326188053794476159516,
    0.354151399451909419671463603538,
    0.529458575234917277706149699996,
    0.701814529939099963837152670310,
    0.849379320441106676048309202301,
    0.953326450056359788767379678514
]

log_gauss_weights = [
    0.16441660472800286831472568326,
    0.23752561002330602050134856196,
    0.22684198443191912636887804029,
    0.17575407900607024498880562120,
    0.11292403024675905185500044208,
    0.057872210717782072399527967294,
    0.020979073742132978043461524115,
    0.0036864071040276190133523212764
]

def compute_greens_function_split(r, r_target, z, z_target, alpha):
    """
    Computes the Green's function G, split into regular and logarithmic parts.
    """
    zd = z - z_target
    a = r**2 + r_target**2 + zd**2
    b = 2 * r * r_target
    m = (4 * r * r_target) / ((r + r_target)**2 + zd**2)
    m1 = 1 - m

    if m < 0 or m >= 1:
        raise ValueError("The modulus m is out of the valid range [0, 1).")

    # Compute elliptic integral K(m)
    K_m = ellipk(m)
    K_star = K_m - b0 * np.log(1 / m1)
    sqrt_ab = np.sqrt(a + b)
    
    G_regular = (4 / sqrt_ab) * K_star
    G_logarithmic = -(4 * b0 / sqrt_ab) * np.log(1 / m1)
    
    return G_regular, G_logarithmic

def compute_greens_function_derivative_split(r, r_target, z, z_target, nr, nz, alpha):
    """
    Computes the directional derivative of the Green's function (∂G/∂n), split into regular and logarithmic parts.
    """
    zd = z - z_target
    a = r**2 + r_target**2 + zd**2
    b = 2 * r * r_target
    m = (4 * r * r_target) / ((r + r_target)**2 + zd**2)
    m1 = 1 - m

    if m < 0 or m >= 1:
        raise ValueError("The modulus m is out of the valid range [0, 1).")

    # Compute elliptic integrals
    K_m = ellipk(m)
    E_m = ellipe(m)
    K_star = K_m - b0 * np.log(1 / m1)
    sqrt_ab = np.sqrt(a + b)

    term1 = 2 * nr / sqrt_ab
    term2 = r * (nr * (r - r_target) + nz * zd) / ((a - b) * sqrt_ab)
    
    DevG_regular = term1 * (E_m - K_star) + term2 * E_m
    DevG_logarithmic = -term1 * b0 * np.log(1 / m1)
    
    return DevG_regular, DevG_logarithmic

def integrate_k1_k2_split(r1, z1, r2, z2, r_target, z_target, nr, nz, alpha):
    """
    Computes [k1] and [k2] using Gaussian quadrature, split into regular and logarithmic parts.
    """
    gauss_points = [
        -0.96028985649753623168,
        -0.79666647741362673959,
        -0.52553240991632898582,
        -0.18343464249564980494,
         0.18343464249564980494,
         0.52553240991632898582,
         0.79666647741362673959,
         0.96028985649753623168
    ]
    gauss_weights = [
        0.10122853629037625915,
        0.22238103445337447054,
        0.31370664587788728734,
        0.36268378337836209114,
        0.36268378337836209114,
        0.31370664587788728734,
        0.22238103445337447054,
        0.10122853629037625915
    ]

    element_length = np.sqrt((z2 - z1)**2 + (r2 - r1)**2)
    scale = element_length / 2.0
    
    k1_regular, k1_logarithmic = 0.0, 0.0
    k2_regular, k2_logarithmic = 0.0, 0.0
    
    for xi, weight in zip(gauss_points, gauss_weights):
        z = z1 * (1 - xi) / 2.0 + z2 * (1 + xi) / 2.0
        r = r1 * (1 - xi) / 2.0 + r2 * (1 + xi) / 2.0
        
        G_regular, G_logarithmic = compute_greens_function_split(r, r_target, z, z_target, alpha)
        dG_regular, dG_logarithmic = compute_greens_function_derivative_split(r, r_target, z, z_target, nr, nz, alpha)
        
        N1 = (1 - xi) / 2.0
        N2 = (1 + xi) / 2.0
        
        k1_regular += weight * (N1 * dG_regular)
        k1_logarithmic += weight * (N1 * dG_logarithmic)
        k2_regular += weight * (N2 * G_regular)
        k2_logarithmic += weight * (N2 * G_logarithmic)
    
    k1_regular *= scale
    k1_logarithmic *= scale
    k2_regular *= scale
    k2_logarithmic *= scale
    
    return k1_regular + k1_logarithmic, k2_regular + k2_logarithmic

# Example usage
r1, z1 = 2.0, 5.0
r2, z2 = 3.0, 6.0
r_target, z_target = 2.0, 5.0
nr, nz = 1.0, 0.0
alpha = np.pi / 4

k1, k2 = integrate_k1_k2_split(r1, z1, r2, z2, r_target, z_target, nr, nz, alpha)
print(f"K1: {k1}, K2: {k2}")

K1: 0.6292205282056171, K2: -3.8767673152121778
